In [5]:
import os
import time

In [6]:
#Size of the file
os.path.getsize("C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/LI-Medium_Trans.csv")

2976436978

In [7]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv("C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/LI-Medium_Trans.csv")
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.009280204772949219 sec


In [8]:
import pandas as pd
start = time.time()
df = pd.read_csv("C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/LI-Medium_Trans.csv")
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  58.559502363204956 sec


In [9]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv("C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/LI-Medium_Trans.csv")
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2024-07-12 21:49:46,822	INFO worker.py:1788 -- Started a local Ray instance.
(pid=31684) c:\Users\biswa\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
(pid=31684)   from pandas.core import (


Read csv with modin and ray:  54.58161640167236 sec


Here Dask is better than Pandas, Modin and Ray, with the least reading time of 0.01sec

In [10]:
from dask import dataframe as dd
df = dd.read_csv("C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/LI-Medium_Trans.csv",delimiter=',')

In [11]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, Timestamp to Is Laundering
dtypes: object(6), float64(2), int64(3)

In [12]:
#No. of Rows
len(df.index)

31251483

In [13]:
#No, of Columns
len(df.columns)

11

In [14]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [15]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [16]:
data=df.columns
data

Index(['Timestamp', 'FromBank', 'Account', 'ToBank', 'Account.1',
       'AmountReceived', 'ReceivingCurrency', 'AmountPaid', 'PaymentCurrency',
       'PaymentFormat', 'IsLaundering'],
      dtype='object')

Validation

In [1]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

c:\Users\biswa\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
%%writefile utility.py
import logging
import yaml
import pandas as pd
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)
            return None

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df, table_config):
    '''
    Replace whitespaces in the column and standardize column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("Column name and column length validation passed")
        return 1
    else:
        print("Column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following file columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'DF columns: {df.columns}')
        logging.info(f'Expected columns: {expected_col}')
        return 0


Overwriting utility.py


In [36]:
%%writefile file.yaml
file_type: csv
dataset_name: file
file_name: LI-Medium_Trans
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
   - Timestamp
   - From Bank
   - Account
   - To Bank
   - Account.1
   - Amount Received
   - Receiving Currency
   - Amount Paid
   - Payment Currency
   - Payment Format
   - Is Laundering

Overwriting file.yaml


In [3]:
# Reading config file

import utility as util
config_data = util.read_config_file("file.yaml")
config_data
     

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'LI-Medium_Trans',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Timestamp',
  'From Bank',
  'Account',
  'To Bank',
  'Account.1',
  'Amount Received',
  'Receiving Currency',
  'Amount Paid',
  'Payment Currency',
  'Payment Format',
  'Is Laundering']}

In [4]:
config_data['file_type']

'csv'

In [5]:
config_data['inbound_delimiter']


','

In [21]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/" + config_data['file_name'] + f'.{file_type}'

In [19]:
config_data['inbound_delimiter']

','

In [22]:
import pandas as pd
df = pd.read_csv(source_file,delimiter=config_data['inbound_delimiter'])
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:15,20,800104D70,20,800104D70,8095.07,US Dollar,8095.07,US Dollar,Reinvestment,0
1,2022/09/01 00:18,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:23,1208,80010E430,1208,80010E430,2654.22,US Dollar,2654.22,US Dollar,Reinvestment,0
3,2022/09/01 00:19,3203,80010EA80,3203,80010EA80,13284.41,US Dollar,13284.41,US Dollar,Reinvestment,0
4,2022/09/01 00:27,20,800104D20,20,800104D20,9.72,US Dollar,9.72,US Dollar,Reinvestment,0


In [23]:
#validating the header of the file
util.col_header_val(df,config_data)

Column name and column length validation failed
Following file columns are not in the YAML file ['account_1', 'amount_paid', 'amount_received', 'from_bank', 'payment_format', 'receiving_currency', 'is_laundering', 'payment_currency', 'to_bank']
Following YAML columns are not in the file uploaded ['from bank', 'account.1', 'amount received', 'receiving currency', 'to bank', 'payment format', 'payment currency', 'amount paid', 'is laundering']


0

In [24]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

Column name and column length validation failed
Following file columns are not in the YAML file ['account_1', 'amount_paid', 'amount_received', 'from_bank', 'payment_format', 'receiving_currency', 'is_laundering', 'payment_currency', 'to_bank']
Following YAML columns are not in the file uploaded ['from bank', 'account.1', 'amount received', 'receiving currency', 'to bank', 'payment format', 'payment currency', 'amount paid', 'is laundering']
validation failed


In [26]:
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('C:/ads/DataGlacier/week6/LI-Medium_Trans.csv/LI-Medium_Trans.csv', delimiter=",")

# Write csv in gz format in pipe separated text file (|)
df.to_csv('LI-Medium_Trans.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True
          )

['c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\00.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\01.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\02.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\03.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\04.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\05.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\06.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\07.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\08.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\09.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\10.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\11.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\12.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\13.part',
 'c:\\ads\\DataGlacier\\week6\\LI-Medium_Trans.csv.gz\\14.part',
 'c:\\ads\\DataGlacier\\w

In [27]:
# Get file summary
file_size = os.path.getsize('LI-Medium_Trans.csv.gz')
num_rows = len(df)
num_cols = len(df.columns)

# Print file summary
print("File summary:")
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")
print(f"File size: {file_size} bytes")

File summary:
Number of rows: 31251483
Number of columns: 11
File size: 16384 bytes
